Here are the release dates for the Google Pixel flagship phone series you asked about:

Model	Release date
Pixel 6	October 28 2021 
Wikipedia
+1

Pixel 7	October 13 2022 
Wikipedia
+1

Pixel 8	October 12 2023 
Wikipedia
+1

Pixel 9	August 22 2024 
Wikipedia
+1

Pixel 10	August 28 2025

In [4]:
import pandas as pd 

#read pixel_base
data = pd.read_csv("pixel_base.csv")
data.head()

,Page,Title,Price,Sold Date,Link,Image Link,Storage,Condition,PartsOnly,Model,Pixel_6,Pixel_7,Pixel_8,Pixel_9,Pixel_10
0,1,Google Pixel 6 5G UW 256GB G9S9B Verizon/Unloc...,$195.49,2025-07-23,https://www.ebay.com/itm/267327711422,https://i.ebayimg.com/images/g/9FQAAOSwxOBicD9...,256 GB,Excellent,False,Pixel 6,True,NaN,NaN,NaN,NaN
1,1,Google Pixel 6 128GB GB7N6 CDMA/GSM Unlocked S...,$161.49,2025-10-18,https://www.ebay.com/itm/267187173848,https://i.ebayimg.com/images/g/9FQAAOSwxOBicD9...,128 GB,Excellent,False,Pixel 6,True,NaN,NaN,NaN,NaN
2,1,"Unlocked Google Pixel 6 GA02910-US 6.4"" 128GB ...",$129.99,2025-10-17,https://www.ebay.com/itm/197701916743,https://i.ebayimg.com/images/g/eO4AAOSwck9hyhr...,128 GB,Unknown,False,Pixel 6,True,NaN,NaN,NaN,NaN
3,1,Google Pixel 6 5G GB7N6 Unlocked AT&T T-Mobile...,$135.99,2025-10-17,https://www.ebay.com/itm/186382972121,https://i.ebayimg.com/images/g/9UwAAOSwRVBizcu...,128 GB,Very Good,False,Pixel 6,True,NaN,NaN,NaN,NaN
4,1,Google Pixel 6 5G GB7N6 Unlocked AT&T T-Mobile...,$126.99,2025-10-16,https://www.ebay.com/itm/204733253931,https://i.ebayimg.com/images/g/9UwAAOSwRVBizcu...,128 GB,Good,False,Pixel 6,True,NaN,NaN,NaN,NaN


In [7]:
#from 
data['Release Date'] = data['Model'].replace({
    "Pixel 6": "2021-10",
    "Pixel 7": "2022-10",
    "Pixel 8": "2023-10",
    "Pixel 9": "2024-08",
    "Pixel 10": "2025-08"
})

In [8]:
import pandas as pd

# Map each model to its release YEAR-MONTH (string)
release_map = {
    "pixel 6": "2021-10",
    "pixel 7": "2022-10",
    "pixel 8": "2023-10",
    "pixel 9": "2024-08",
    "pixel 10": "2025-08"
}

# Assign release date (only year and month)
data['Release Date'] = data['Model'].map(release_map)

# Convert to datetime (default day=1 — but we’ll only use year/month info)
data['Release Date'] = pd.to_datetime(data['Release Date'], format='%Y-%m', errors='coerce')

# Compute age at sale in MONTHS using year and month difference
data['Age_at_Sale'] = (
    (data['Sold Date'].dt.year - data['Release Date'].dt.year) * 12 +
    (data['Sold Date'].dt.month - data['Release Date'].dt.month)
)

# Round or ensure type consistency
data['Age_at_Sale'] = data['Age_at_Sale'].astype(float).round(2)

# Optional: display only month-year text for clarity
data['Release Date (Year-Month)'] = data['Release Date'].dt.strftime('%Y-%m')

# Preview
print(data[['Model', 'Sold Date', 'Release Date (Year-Month)', 'Age_at_Sale']].head())

AttributeError: Can only use .dt accessor with datetimelike values

In [ ]:
import re
import numpy as np
import pandas as pd

# ----------------------------
# Launch prices for Pixel models
# ----------------------------
release_price_map = {
    '6':   {'128': 599.0, '256': 699.0},
    '7':   {'128': 599.0, '256': 699.0},
    '8':   {'128': 699.0, '256': 759.0},
    '9':   {'128': 799.0, '256': 899.0, '512': 1099.0},
    '10':  {'128': 799.0, '256': 899.0, '512': 1099.0},
}

# ----------------------------
# Extract storage
# ----------------------------
def extract_storage_from_row(row):
    # Try explicit columns first
    for col in ['Storage', 'Capacity', 'Memory', 'Storage Capacity']:
        if col in row.index and pd.notna(row[col]):
            s = str(row[col])
            m = re.search(r'(128|256|512)', s)
            if m:
                return m.group(1)
    # Fallback: parse from Title
    title = str(row.get('Title', ''))
    m = re.search(r'\b(128|256|512)\s*(?:GB|G|gb)?\b', title, flags=re.IGNORECASE)
    if m:
        return m.group(1)
    m = re.search(r'(128|256|512)GB', title, flags=re.IGNORECASE)
    if m:
        return m.group(1)
    return None

# ----------------------------
# Extract Pixel model number (6–10)
# ----------------------------
def extract_model_number(row):
    # Check Model field
    model_field = str(row.get('Model', ''))
    m = re.search(r'pixel\s*([6-9]|10)', model_field, flags=re.IGNORECASE)
    if m:
        return m.group(1)
    # Fallback to Title
    title = str(row.get('Title', ''))
    m = re.search(r'pixel\s*([6-9]|10)', title, flags=re.IGNORECASE)
    if m:
        return m.group(1)
    return None

# ----------------------------
# Map to release price
# ----------------------------
def map_first_release_price(row):
    model_num = extract_model_number(row)
    if not model_num:
        return np.nan
    model_num = model_num.strip()
    if model_num not in release_price_map:
        return np.nan
    storage = extract_storage_from_row(row)
    if storage and storage in release_price_map[model_num]:
        return release_price_map[model_num][storage]
    return np.nan

# ----------------------------
# Load data if needed
# ----------------------------
if 'data' not in globals():
    try:
        data = pd.read_csv('Pixel_base.csv')
        data['Sold Date'] = pd.to_datetime(data['Sold Date'], errors='coerce')
    except FileNotFoundError:
        raise FileNotFoundError('Pixel_base.csv not found; please load or create `data`.')

# ----------------------------
# Apply mapping
# ----------------------------
data['First_Release_Price'] = data.apply(map_first_release_price, axis=1)

# ----------------------------
# Quick fill-rate report
# ----------------------------
for model_num, mapping in release_price_map.items():
    model_label = f'Pixel {model_num}'
    total = data[data['Model'].str.contains(model_label, case=False, na=False)].shape[0]
    filled = data[(data['Model'].str.contains(model_label, case=False, na=False)) & (data['First_Release_Price'].notna())].shape[0]
    print(f'{model_label}: filled {filled} / {total}')

# ----------------------------
# Show examples where price was set
# ----------------------------
display(data[data['First_Release_Price'].notna()].head())

# Optionally save to CSV
# data.to_csv('Pixel_base_with_release_price.csv', index=False)


Pixel 6: filled 24 / 24
Pixel 7: filled 64 / 68
Pixel 8: filled 11 / 13
Pixel 9: filled 55 / 56
Pixel 10: filled 0 / 0


,Page,Title,Price,Sold Date,Link,Image Link,Storage,Condition,PartsOnly,Model,Pixel_6,Pixel_7,Pixel_8,Pixel_9,Pixel_10,Release Date,Age_at_Sale,Release Date (Year-Month),First_Release_Price
0,1,Google Pixel 6 5G UW 256GB G9S9B Verizon/Unloc...,$195.49,2025-07-23,https://www.ebay.com/itm/267327711422,https://i.ebayimg.com/images/g/9FQAAOSwxOBicD9...,256 GB,Excellent,False,Pixel 6,True,NaN,NaN,NaN,NaN,NaT,NaN,NaN,699.0
1,1,Google Pixel 6 128GB GB7N6 CDMA/GSM Unlocked S...,$161.49,2025-10-18,https://www.ebay.com/itm/267187173848,https://i.ebayimg.com/images/g/9FQAAOSwxOBicD9...,128 GB,Excellent,False,Pixel 6,True,NaN,NaN,NaN,NaN,NaT,NaN,NaN,599.0
2,1,"Unlocked Google Pixel 6 GA02910-US 6.4"" 128GB ...",$129.99,2025-10-17,https://www.ebay.com/itm/197701916743,https://i.ebayimg.com/images/g/eO4AAOSwck9hyhr...,128 GB,Unknown,False,Pixel 6,True,NaN,NaN,NaN,NaN,NaT,NaN,NaN,599.0
3,1,Google Pixel 6 5G GB7N6 Unlocked AT&T T-Mobile...,$135.99,2025-10-17,https://www.ebay.com/itm/186382972121,https://i.ebayimg.com/images/g/9UwAAOSwRVBizcu...,128 GB,Very Good,False,Pixel 6,True,NaN,NaN,NaN,NaN,NaT,NaN,NaN,599.0
4,1,Google Pixel 6 5G GB7N6 Unlocked AT&T T-Mobile...,$126.99,2025-10-16,https://www.ebay.com/itm/204733253931,https://i.ebayimg.com/images/g/9UwAAOSwRVBizcu...,128 GB,Good,False,Pixel 6,True,NaN,NaN,NaN,NaN,NaT,NaN,NaN,599.0


In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import re

# ---- 0) Filter to Pixel only (uses Model or Title) ----
if 'Model' not in data.columns and 'Title' not in data.columns:
    raise KeyError("Expected a 'Model' or 'Title' column to identify Pixel rows.")

# Normalize / clean column names
data.rename(columns=str.strip, inplace=True)

pixel_mask = (
    data.get('Model', pd.Series('', index=data.index)).astype(str).str.contains(r'\bpixel\b', flags=re.I, regex=True)
    | data.get('Title', pd.Series('', index=data.index)).astype(str).str.contains(r'\bpixel\b', flags=re.I, regex=True)
)

pixel_df = data.loc[pixel_mask].copy()
if pixel_df.empty:
    raise ValueError("No rows matched Pixel devices. Check that 'Model' or 'Title' contains 'Pixel'.")

# ---- 1) Clean & coerce to numeric (Pixel subset) ----
# Age: keep the first number (handles "12", "12.5", "12 mo")
x = pd.to_numeric(
    pixel_df['Age_at_Sale'].astype(str).str.extract(r'([+-]?\d*\.?\d+)')[0],
    errors='coerce'
).to_numpy(dtype=float)

# Price: strip $ and commas then convert
y = pd.to_numeric(
    pixel_df['Price'].astype(str).str.replace(r'[$,]', '', regex=True),
    errors='coerce'
).to_numpy(dtype=float)

# ---- 2) Mask invalids after coercion ----
m = np.isfinite(x) & np.isfinite(y)
x = x[m]
y = y[m]

# ---- 3) Guard against too-small data ----
if x.size < 6:
    raise ValueError(f"Not enough clean Pixel points for a 5th-degree fit (got {x.size}).")

# ---- 4) Fit & plot (consider lowering degree if overfitting) ----
deg = 3
coeffs = np.polyfit(x, y, deg)
poly_eq = np.poly1d(coeffs)

x_line = np.linspace(x.min(), x.max(), 200)
y_line = poly_eq(x_line)

plt.scatter(x, y, s=15, alpha=0.6, label='Pixel data')
plt.plot(x_line, y_line, linewidth=2, label=f'{deg}th-degree fit')
plt.xlabel('Age at Sale (Months)')
plt.ylabel('Price ($)')
plt.title('Google Pixel Price Depreciation Over Time')
plt.legend()
plt.tight_layout()
plt.show()


ValueError: Not enough clean Pixel points for a 5th-degree fit (got 0).

In [ ]:
import re
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from packaging import version
import sklearn

# ----------------------------
# CONFIG (Pixel)
# ----------------------------
# Pixel storage tiers commonly used in your mapping
ALLOWED_STORAGE = [128, 256, 512]
ALLOWED_CONDITIONS = ['Good', 'Very Good', 'Excellent']
CONDITION_BASELINE = None  # keep all dummies

# Pixel launch prices (US) by storage
PIXEL_RELEASE_PRICE_MAP = {
    '6':  {'128': 599.0, '256': 699.0},
    '7':  {'128': 599.0, '256': 699.0},
    '8':  {'128': 699.0, '256': 759.0},
    '9':  {'128': 799.0, '256': 899.0, '512': 1099.0},
    '10': {'128': 799.0, '256': 899.0, '512': 1099.0},
}

# ----------------------------
# Helpers to identify Pixel, storage, model number, and map launch price
# ----------------------------
def _extract_storage(row):
    # Try explicit columns first
    for col in ['Storage', 'Capacity', 'Memory', 'Storage Capacity']:
        if col in row and pd.notna(row[col]):
            m = re.search(r'(128|256|512)', str(row[col]))
            if m: return m.group(1)
    # Fallback: parse Title
    title = str(row.get('Title', ''))
    m = re.search(r'\b(128|256|512)\s*(?:GB|G|gb)?\b', title, flags=re.IGNORECASE)
    if m: return m.group(1)
    m = re.search(r'(128|256|512)GB', title, flags=re.IGNORECASE)
    return m.group(1) if m else None

def _extract_pixel_model_num(row):
    # From Model
    m = re.search(r'pixel\s*(6|7|8|9|10)\b', str(row.get('Model', '')), flags=re.IGNORECASE)
    if m: return m.group(1)
    # From Title
    m = re.search(r'pixel\s*(6|7|8|9|10)\b', str(row.get('Title', '')), flags=re.IGNORECASE)
    if m: return m.group(1)
    return None

def _map_first_release_price_pixel(row):
    mdl = _extract_pixel_model_num(row)
    if not mdl or mdl not in PIXEL_RELEASE_PRICE_MAP:
        return np.nan
    sto = _extract_storage(row)
    if sto and sto in PIXEL_RELEASE_PRICE_MAP[mdl]:
        return PIXEL_RELEASE_PRICE_MAP[mdl][sto]
    return np.nan

# ----------------------------
# INPUT: assume `data` DataFrame exists
# ----------------------------
data = data.rename(columns=str.strip).copy()
data['Sold Date'] = pd.to_datetime(data['Sold Date'], errors='coerce')

# Filter to Pixel rows via Model or Title
pixel_mask = (
    data.get('Model', pd.Series('', index=data.index)).astype(str).str.contains(r'\bpixel\b', flags=re.I, regex=True) |
    data.get('Title', pd.Series('', index=data.index)).astype(str).str.contains(r'\bpixel\b', flags=re.I, regex=True)
)
data = data.loc[pixel_mask].copy()
if data.empty:
    raise ValueError("No Pixel devices found in `data` (looked in Model/Title).")

# 1) Age_at_Sale (months)
if 'Age_at_Sale' not in data.columns:
    data['Release Date'] = pd.to_datetime(data.get('Release Date'), errors='coerce')
    data['Age_at_Sale'] = (data['Sold Date'] - data['Release Date']).dt.days / 30.44

# 2) Clean numeric money columns
def _clean_money(s):
    return pd.to_numeric(
        pd.Series(s, dtype="object").astype(str).str.replace(r'[$,]', '', regex=True),
        errors='coerce'
    )

data['Price_clean'] = _clean_money(data['Price'])

# Fill/ensure First_Release_Price from Pixel map if missing
if 'First_Release_Price' not in data.columns:
    data['First_Release_Price'] = np.nan
missing_mask = data['First_Release_Price'].isna()
if missing_mask.any():
    data.loc[missing_mask, 'First_Release_Price'] = data.loc[missing_mask].apply(_map_first_release_price_pixel, axis=1)

data['First_Release_Price_clean'] = _clean_money(data['First_Release_Price'])

# 3) Filter valid rows
mask = (
    data['Age_at_Sale'].notna() &
    data['Price_clean'].notna() &
    data['First_Release_Price_clean'].notna() &
    (data['First_Release_Price_clean'] > 0) &
    data['Sold Date'].notna()
)
df = data.loc[mask].copy()
if df.empty:
    raise ValueError("After cleaning, no valid Pixel rows remained (check Age/Price/Launch price/Sold Date).")

# 4) Target: resale ratio
df['Price_ratio'] = df['Price_clean'] / df['First_Release_Price_clean']

# 5) Storage (parse from Storage or Title)
if 'Storage' in df.columns:
    storage_src = df['Storage'].astype(str)
else:
    storage_src = df.get('Title', pd.Series(index=df.index, dtype=str)).astype(str)

df['Storage_num'] = pd.to_numeric(storage_src.str.extract(r'(\d+)', expand=False), errors='coerce')
df = df.loc[df['Storage_num'].isin(ALLOWED_STORAGE)].copy()
df['Storage_num'] = df['Storage_num'].astype(int)

# 6) Condition cleanup
df['Condition'] = df.get('Condition', pd.Series(index=df.index, dtype=object)).astype(str).str.strip().str.title()
remove_conditions = {'Fair', 'Unknown', 'Unspecified', 'Na', 'Nan', 'None', ''}
df = df.loc[~df['Condition'].isin(remove_conditions)].copy()
df = df.loc[df['Condition'].isin(ALLOWED_CONDITIONS)].copy()
df['Condition'] = pd.Categorical(df['Condition'], categories=ALLOWED_CONDITIONS, ordered=True)

# 7) Encoders (keep all dummies)
skver = version.parse(sklearn.__version__)

if skver >= version.parse("1.2"):
    enc_cond = OneHotEncoder(categories=[ALLOWED_CONDITIONS], drop=None, sparse_output=False, handle_unknown='ignore')
    enc_sto  = OneHotEncoder(categories=[ALLOWED_STORAGE],  drop=None, sparse_output=False, handle_unknown='ignore')
else:
    enc_cond = OneHotEncoder(categories=[ALLOWED_CONDITIONS], drop=None, sparse=False, handle_unknown='ignore')
    enc_sto  = OneHotEncoder(categories=[ALLOWED_STORAGE],  drop=None, sparse=False, handle_unknown='ignore')

cond_mat  = enc_cond.fit_transform(df[['Condition']])
cond_cols = enc_cond.get_feature_names_out(['Condition'])
cond_df   = pd.DataFrame(cond_mat, columns=cond_cols, index=df.index)

sto_mat  = enc_sto.fit_transform(df[['Storage_num']])
sto_cols = enc_sto.get_feature_names_out(['Storage_num'])
sto_df   = pd.DataFrame(sto_mat, columns=sto_cols, index=df.index)

# 8) Assemble X/y
X = pd.concat([df[['Age_at_Sale']].apply(pd.to_numeric, errors='coerce'), sto_df, cond_df], axis=1)
y = pd.to_numeric(df['Price_ratio'], errors='coerce')

good = X.notna().all(axis=1) & y.notna()
X = X.loc[good]
y = y.loc[good]

if len(y) < 10:
    raise ValueError(f"Too few clean Pixel samples to train (got {len(y)}).")

# 9) Fit model
model = LinearRegression()
model.fit(X, y)
X_COLUMNS = X.columns

# 10) Report
print("=== Pixel Model Fit (Age + categorical Storage + Condition) ===")
print("Samples used:", len(y))
print("Intercept:", model.intercept_)
print("Top coefficients (by magnitude):")
coef_series = pd.Series(model.coef_, index=X.columns).abs().sort_values(ascending=False).head(12)
for name in coef_series.index:
    print(f"  {name}: {model.coef_[X.columns.get_loc(name)]:.6f}")

# 11) Predictor
def predict_price(age_months, storage_gb, condition, launch_price):
    storage_gb = int(storage_gb)
    if storage_gb not in set(ALLOWED_STORAGE):
        raise ValueError(f"storage_gb must be one of {ALLOWED_STORAGE}; got {storage_gb}")
    condition = str(condition).strip().title()
    if condition not in set(ALLOWED_CONDITIONS):
        raise ValueError(f"condition must be one of {ALLOWED_CONDITIONS}; got {condition}")

    base = pd.DataFrame({
        'Age_at_Sale': [float(age_months)],
        'Storage_num': [storage_gb],
        'Condition': [condition]
    })

    cond_new = pd.DataFrame(enc_cond.transform(base[['Condition']]), columns=cond_cols, index=base.index)
    sto_new  = pd.DataFrame(enc_sto.transform(base[['Storage_num']]), columns=sto_cols, index=base.index)
    X_new = pd.concat([base[['Age_at_Sale']], sto_new, cond_new], axis=1)
    X_new = X_new.reindex(columns=X_COLUMNS, fill_value=0)

    ratio = float(model.predict(X_new))
    return ratio * float(launch_price)

# ----------------------------
# Example predictions
# ----------------------------
# Pixel 9, 128GB, Excellent, using mapped launch price if desired:
px9_launch = PIXEL_RELEASE_PRICE_MAP['9']['128']
price_usd = predict_price(age_months=18, storage_gb=128, condition='Excellent', launch_price=px9_launch)
print(f"Predicted Pixel 9 resale at 18 mo (128GB, Excellent): ${price_usd:.2f}")


ValueError: After cleaning, no valid Pixel rows remained (check Age/Price/Launch price/Sold Date).

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import re

# =========================
# CONFIG
# =========================
HORIZON_MONTHS = 12              # how many months ahead to forecast
TARGET_MODEL_REGEX = r'pixel'    # filter which Pixel line to use for release date inference (regex on Model/Title)
MANUAL_RELEASE_DATE = None       # e.g., "2024-10-12" or None to auto-infer

MODEL_TYPE = "loglinear"         # "loglinear" (recommended) or "poly"
POLY_DEG = 5                     # only used if MODEL_TYPE == "poly"
BUFFER_MONTHS = 3                # allow extrapolation only this many months beyond observed max x
MIN_POINTS_FOR_FIT = 6           # minimal usable data points (after cleaning)
PRICE_MIN_FLOOR = 0              # final clipping floor for predictions (keep at 0 or a small >0 floor)
DROP_PRICE_BELOW = 10            # treat anything below this as junk / accessories / errors
DROP_PRICE_TOP_PCTL = 99         # cap outliers for robust fit

# If you're NOT running this where `data` already exists, you can load a CSV:
CSV_PATH = None  # e.g., "my_export.csv" or leave None to use an existing `data` DataFrame in memory


# =========================
# HELPERS
# =========================
def _coerce_dt(s):
    return pd.to_datetime(s, errors='coerce')

def _ensure_dataframe():
    global data
    if CSV_PATH is not None:
        return pd.read_csv(CSV_PATH)
    if 'data' not in globals():
        raise NameError("No global DataFrame named `data` and CSV_PATH is None. "
                        "Either set CSV_PATH to a file or define `data` first.")
    return data.copy()

# =========================
# LOAD & BASIC FILTERING
# =========================
data = _ensure_dataframe()

# Normalize / clean column names
data.rename(columns=lambda c: str(c).strip(), inplace=True)

# Basic column presence checks
if 'Model' not in data.columns and 'Title' not in data.columns:
    raise KeyError("Expected a 'Model' or 'Title' column to identify Pixel rows.")
if 'Age_at_Sale' not in data.columns:
    raise KeyError("Expected an 'Age_at_Sale' column.")
if 'Price' not in data.columns:
    raise KeyError("Expected a 'Price' column.")

# Filter to Pixel only (using Model or Title)
pixel_mask = (
    data.get('Model', pd.Series('', index=data.index)).astype(str).str.contains(r'\bpixel\b', flags=re.I, regex=True)
    | data.get('Title', pd.Series('', index=data.index)).astype(str).str.contains(r'\bpixel\b', flags=re.I, regex=True)
)
pixel_df = data.loc[pixel_mask].copy()
if pixel_df.empty:
    raise ValueError("No rows matched Pixel devices. Check that 'Model' or 'Title' contains 'Pixel'.")

# Optional: narrow the subset for release date inference to a specific Pixel generation
target_mask = (
    pixel_df.get('Model', pd.Series('', index=pixel_df.index)).astype(str).str.contains(TARGET_MODEL_REGEX, flags=re.I, regex=True)
    | pixel_df.get('Title', pd.Series('', index=pixel_df.index)).astype(str).str.contains(TARGET_MODEL_REGEX, flags=re.I, regex=True)
)
sub_df = pixel_df.loc[target_mask] if target_mask.any() else pixel_df

# =========================
# CLEAN & COERCE NUMERIC
# =========================
# Age: keep the first number (handles "12", "12.5", "12 mo")
x_raw = pd.to_numeric(
    pixel_df['Age_at_Sale'].astype(str).str.extract(r'([+-]?\d*\.?\d+)')[0],
    errors='coerce'
).to_numpy(dtype=float)

# Price: strip $ and commas then convert
y_raw = pd.to_numeric(
    pixel_df['Price'].astype(str).str.replace(r'[$,]', '', regex=True),
    errors='coerce'
).to_numpy(dtype=float)

# Basic validity mask
m = np.isfinite(x_raw) & np.isfinite(y_raw)
x = x_raw[m]
y = y_raw[m]

# Additional cleaning: drop suspiciously low prices and extreme outliers
if y.size == 0:
    raise ValueError("No valid Pixel price data after initial coercion.")
upper_cap = np.nanpercentile(y, DROP_PRICE_TOP_PCTL)
valid_price = (y > DROP_PRICE_BELOW) & (y <= upper_cap)
x = x[valid_price]
y = y[valid_price]

# Guard against too-small data
if x.size < MIN_POINTS_FOR_FIT:
    raise ValueError(f"Not enough clean Pixel points for a fit (got {x.size}, need ≥ {MIN_POINTS_FOR_FIT}).")

# =========================
# FIT MODEL (LOG-LINEAR DEFAULT)
# =========================
if MODEL_TYPE.lower() == "loglinear":
    # enforce positivity for log
    y_pos = np.where(y <= 0, np.nan, y)
    m2 = np.isfinite(x) & np.isfinite(y_pos)
    x_fit = x[m2]
    y_fit = y_pos[m2]
    if x_fit.size < MIN_POINTS_FOR_FIT:
        raise ValueError(f"Not enough positive-price points for log fit (got {x_fit.size}).")

    coeffs = np.polyfit(x_fit, np.log(y_fit), 1)  # log(y) = a*x + b
    def predict_fn(xv):
        return np.exp(np.poly1d(coeffs)(xv))  # strictly > 0 before clipping

    fit_label = "Log-linear fit"
    # For drawing the smooth curve
    x_line = np.linspace(np.nanmin(x_fit), np.nanmax(x_fit), 300)
    y_line = predict_fn(x_line)

elif MODEL_TYPE.lower() == "poly":
    deg = int(POLY_DEG)
    if x.size <= deg:
        raise ValueError(f"Need more points than degree for poly fit (deg={deg}, points={x.size}).")
    coeffs = np.polyfit(x, y, deg)
    poly_eq = np.poly1d(coeffs)

    def predict_fn(xv):
        return poly_eq(xv)

    fit_label = f"{deg}° polynomial fit"
    x_line = np.linspace(np.nanmin(x), np.nanmax(x), 300)
    y_line = predict_fn(x_line)
else:
    raise ValueError("MODEL_TYPE must be 'loglinear' or 'poly'.")

# =========================
# INFER "TODAY" AGE IN MONTHS
# =========================
today = pd.Timestamp.today().normalize()

# Try explicit Release Date first (from the sub_df matching TARGET_MODEL_REGEX)
if MANUAL_RELEASE_DATE:
    rel_date = _coerce_dt(MANUAL_RELEASE_DATE)
else:
    rel_date = None
    if 'Release Date' in sub_df.columns:
        # Use median (robust) instead of min to avoid an outlier making age_today too large
        rel_date = _coerce_dt(sub_df['Release Date']).dropna()
        rel_date = rel_date.median() if not rel_date.empty else None

# Fallback: estimate release date = median(Sold Date - Age_at_Sale * 30.44 days)
if pd.isna(rel_date) or rel_date is None:
    sold_dates = _coerce_dt(sub_df.get('Sold Date'))
    ages = pd.to_numeric(
        sub_df.get('Age_at_Sale', pd.Series(index=sub_df.index)).astype(str).str.extract(r'([+-]?\d*\.?\d+)')[0],
        errors='coerce'
    )
    if sold_dates is not None and ages is not None:
        est_release = sold_dates - pd.to_timedelta(ages * 30.44, unit='D')
        est_release = est_release.dropna()
        rel_date = est_release.median() if not est_release.empty else None

if pd.isna(rel_date) or rel_date is None:
    raise ValueError("Could not infer release date; set MANUAL_RELEASE_DATE = 'YYYY-MM-DD'.")

# Compute today's age in months since release
age_today = (today - rel_date).days / 30.44

# =========================
# FORECAST HORIZON (CLAMP TO SUPPORT)
# =========================
max_supported_age = np.nanmax(x) + BUFFER_MONTHS
end_age = min(age_today + HORIZON_MONTHS, max_supported_age)
if end_age <= age_today:
    # If you're already beyond observed support (+ buffer), just give a tiny 0-1 month lookahead
    end_age = age_today + 1.0

age_future = np.linspace(age_today, end_age, int(round(end_age - age_today)) + 1)
price_future = predict_fn(age_future)
# Clip to non-negative (safety net)
price_future = np.clip(price_future, PRICE_MIN_FLOOR, None)

# =========================
# PLOTTING
# =========================
plt.figure(figsize=(10, 6))
plt.scatter(x, y, s=15, alpha=0.6, label='Pixel data')
plt.plot(x_line, y_line, linewidth=2, label=fit_label)

# Vertical line for "today"
plt.axvline(age_today, linestyle='--', linewidth=1)
y_text_level = np.nanmax(np.r_[y_line, y]) if np.isfinite(np.r_[y_line, y]).any() else np.nanmax(y)
plt.text(age_today, y_text_level, f" today\n({today.date()})", rotation=90, va='top', ha='left')

# Forecast segment
plt.plot(age_future, price_future, linewidth=3, label=f'Forecast next ≤ {HORIZON_MONTHS} mo (clamped)')

# Extrapolation warning if we clamped
if age_today + HORIZON_MONTHS > max_supported_age:
    plt.title('Google Pixel Price: Today and Next Months (⚠ extrapolation limited)')
else:
    plt.title('Google Pixel Price: Today and Next Months')

plt.xlabel('Months since release')
plt.ylabel('Price ($)')
plt.legend()
plt.tight_layout()
plt.show()

# =========================
# PRINT PREDICTIONS
# =========================
pred_df = pd.DataFrame({
    'Month Ahead': np.arange(0, len(age_future), 1),
    'Age (mo)': np.round(age_future, 2),
    'Predicted Price ($)': np.round(price_future, 2)
})
print(f"Release date used: {rel_date.date()}, today: {today.date()}, age_today ≈ {age_today:.2f} months")
if age_today + HORIZON_MONTHS > max_supported_age:
    print(f"⚠ Forecast was clamped to observed support (max x ≈ {np.nanmax(x):.1f} mo + {BUFFER_MONTHS} mo buffer).")
print(pred_df.to_string(index=False))


ValueError: No valid Pixel price data after initial coercion.